## Artifical Neural Network Using Pytorch
- Advance House Price Prediction Using Pytorch- Tabular Dataset
- https://docs.fast.ai/tabular.html
- https://www.fast.ai/2018/04/29/categorical-embeddings/
- https://www.fast.ai/2018/04/29/categorical-embeddings/
- https://yashuseth.blog/2018/07/22/pytorch-neural-network-for-tabular-data-with-categorical-embeddings/
1. Category Embedding


In [1]:
#Lets import the libraries -
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

C:\Users\om\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Lets load the dataset -
df=pd.read_csv('C:/Users/om/Desktop/PyTorch/houseprice.csv',usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea","Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,YearBuilt,1stFlrSF,2ndFlrSF,SalePrice
0,60,RL,65.0,8450,Pave,Reg,2003,856,854,208500
1,20,RL,80.0,9600,Pave,Reg,1976,1262,0,181500
2,60,RL,68.0,11250,Pave,IR1,2001,920,866,223500
3,70,RL,60.0,9550,Pave,IR1,1915,961,756,140000
4,60,RL,84.0,14260,Pave,IR1,2000,1145,1053,250000


In [3]:
df.shape

(1201, 10)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   object 
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   object 
 5   LotShape     1201 non-null   object 
 6   YearBuilt    1201 non-null   int64  
 7   1stFlrSF     1201 non-null   int64  
 8   2ndFlrSF     1201 non-null   int64  
 9   SalePrice    1201 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 103.2+ KB


In [5]:
#Lets showcase all the unique values from all the columns at a time -
for i in df.columns:
    print("Column name:'{}' and Unique values:{}".format(i,len(df[i].unique())))

Column name:'MSSubClass' and Unique values:15
Column name:'MSZoning' and Unique values:5
Column name:'LotFrontage' and Unique values:110
Column name:'LotArea' and Unique values:869
Column name:'Street' and Unique values:2
Column name:'LotShape' and Unique values:4
Column name:'YearBuilt' and Unique values:112
Column name:'1stFlrSF' and Unique values:678
Column name:'2ndFlrSF' and Unique values:368
Column name:'SalePrice' and Unique values:597


In [6]:
#Lets convert YearBuilt column to Total_years -
import datetime
df['Total_years'] = datetime.datetime.now().year - df['YearBuilt']
df.drop('YearBuilt',axis=1,inplace=True)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,Total_years
0,60,RL,65.0,8450,Pave,Reg,856,854,208500,19
1,20,RL,80.0,9600,Pave,Reg,1262,0,181500,46
2,60,RL,68.0,11250,Pave,IR1,920,866,223500,21
3,70,RL,60.0,9550,Pave,IR1,961,756,140000,107
4,60,RL,84.0,14260,Pave,IR1,1145,1053,250000,22


In [7]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', '1stFlrSF', '2ndFlrSF', 'SalePrice', 'Total_years'],
      dtype='object')

In [8]:
#Lets select Categorical and Output features -
cat_features = ['MSSubClass','MSZoning','Street','LotShape']
out_features = 'SalePrice'

In [9]:
#Lets apply LabelEncoder to all categorical veriables in single shot -
from sklearn.preprocessing import LabelEncoder
le={}
for feature in cat_features:
    le[feature] = LabelEncoder()
    df[feature] = le[feature].fit_transform(df[feature])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,SalePrice,Total_years
0,5,3,65.0,8450,1,3,856,854,208500,19
1,0,3,80.0,9600,1,3,1262,0,181500,46
2,5,3,68.0,11250,1,0,920,866,223500,21
3,6,3,60.0,9550,1,0,961,756,140000,107
4,5,3,84.0,14260,1,0,1145,1053,250000,22


In [10]:
#Lets Convert Categorical features into Numpy array form -
cat_features=np.stack([df['MSSubClass'],df['MSZoning'],df['Street'],df['LotShape']],1)
cat_features

array([[5, 3, 1, 3],
       [0, 3, 1, 3],
       [5, 3, 1, 0],
       ...,
       [6, 3, 1, 3],
       [0, 3, 1, 3],
       [0, 3, 1, 3]], dtype=int64)

In [11]:
#Lets Convert Categorical features (Numpy from) into Tensors form -
cat_features = torch.tensor(cat_features,dtype=torch.int64)
cat_features

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        ...,
        [6, 3, 1, 3],
        [0, 3, 1, 3],
        [0, 3, 1, 3]])

In [12]:
#Lets Select Continous Features -
cont_features=[]
for i in df.columns:
    if i in ['MSSubClass','MSZoning','Street','LotShape','SalePrice']:
        pass
    else:
        cont_features.append(i)
cont_features   

['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'Total_years']

In [13]:
#Lets convert continous features into NumPy form and the into Tensor form -
cont_values=np.stack([df[i].values for i in cont_features],axis=1)
cont_values=torch.tensor(cont_values,dtype=torch.float)
cont_values

tensor([[   65.,  8450.,   856.,   854.,    19.],
        [   80.,  9600.,  1262.,     0.,    46.],
        [   68., 11250.,   920.,   866.,    21.],
        ...,
        [   66.,  9042.,  1188.,  1152.,    81.],
        [   68.,  9717.,  1078.,     0.,    72.],
        [   75.,  9937.,  1256.,     0.,    57.]])

In [14]:
cont_values.dtype

torch.float32

In [15]:
#Lets select Dependent features -
y=torch.tensor(df['SalePrice'].values,dtype=torch.float).reshape(-1,1)
y

tensor([[208500.],
        [181500.],
        [223500.],
        ...,
        [266500.],
        [142125.],
        [147500.]])

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1201 non-null   int64  
 1   MSZoning     1201 non-null   int32  
 2   LotFrontage  1201 non-null   float64
 3   LotArea      1201 non-null   int64  
 4   Street       1201 non-null   int32  
 5   LotShape     1201 non-null   int32  
 6   1stFlrSF     1201 non-null   int64  
 7   2ndFlrSF     1201 non-null   int64  
 8   SalePrice    1201 non-null   int64  
 9   Total_years  1201 non-null   int64  
dtypes: float64(1), int32(3), int64(6)
memory usage: 89.1 KB


In [17]:
#Lets shapes of the veriables -
print(cat_features.shape)
print(cont_values.shape)
print(y.shape)

torch.Size([1201, 4])
torch.Size([1201, 5])
torch.Size([1201, 1])


In [18]:
#Lets Embedd the size for Categorical columns -
cat_dims=[len(df[col].unique()) for col in ["MSSubClass", "MSZoning", "Street", "LotShape"]]
cat_dims

[15, 5, 2, 4]

In [19]:
#Lets Specify the Embedding Dimensions -
embedding_dim = [(x, min(50, (x+1) //2)) for x in cat_dims]
embedding_dim

[(15, 8), (5, 3), (2, 1), (4, 2)]

In [20]:
#Lets Create an object for the algorithm -
import torch.nn as nn
import torch.nn.functional as F
embed_rep=nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
embed_rep

ModuleList(
  (0): Embedding(15, 8)
  (1): Embedding(5, 3)
  (2): Embedding(2, 1)
  (3): Embedding(4, 2)
)

In [21]:
cat_features

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        ...,
        [6, 3, 1, 3],
        [0, 3, 1, 3],
        [0, 3, 1, 3]])

In [22]:
cat_features[:4]

tensor([[5, 3, 1, 3],
        [0, 3, 1, 3],
        [5, 3, 1, 0],
        [6, 3, 1, 0]])

In [23]:
pd.set_option('display.max_rows',500)
embedding_val=[]
for i,e in enumerate(embed_rep):
    embedding_val.append(e(cat_features[:,i]))
embedding_val

[tensor([[-0.4052, -0.3164, -0.8864,  ..., -0.5591, -0.1521,  0.7710],
         [ 0.9767, -0.4553, -0.2691,  ...,  1.4418, -0.2409, -0.0517],
         [-0.4052, -0.3164, -0.8864,  ..., -0.5591, -0.1521,  0.7710],
         ...,
         [-1.0329,  0.8601, -1.1387,  ...,  0.1928, -0.7213, -0.5049],
         [ 0.9767, -0.4553, -0.2691,  ...,  1.4418, -0.2409, -0.0517],
         [ 0.9767, -0.4553, -0.2691,  ...,  1.4418, -0.2409, -0.0517]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[-0.3592,  0.6789,  0.3819],
         [-0.3592,  0.6789,  0.3819],
         [-0.3592,  0.6789,  0.3819],
         ...,
         [-0.3592,  0.6789,  0.3819],
         [-0.3592,  0.6789,  0.3819],
         [-0.3592,  0.6789,  0.3819]], grad_fn=<EmbeddingBackward0>),
 tensor([[0.2960],
         [0.2960],
         [0.2960],
         ...,
         [0.2960],
         [0.2960],
         [0.2960]], grad_fn=<EmbeddingBackward0>),
 tensor([[ 0.2319, -0.4783],
         [ 0.2319, -0.4783],
         [ 1.1340, -0.2285],

In [24]:
z=torch.cat(embedding_val,1)
z

tensor([[-0.4052, -0.3164, -0.8864,  ...,  0.2960,  0.2319, -0.4783],
        [ 0.9767, -0.4553, -0.2691,  ...,  0.2960,  0.2319, -0.4783],
        [-0.4052, -0.3164, -0.8864,  ...,  0.2960,  1.1340, -0.2285],
        ...,
        [-1.0329,  0.8601, -1.1387,  ...,  0.2960,  0.2319, -0.4783],
        [ 0.9767, -0.4553, -0.2691,  ...,  0.2960,  0.2319, -0.4783],
        [ 0.9767, -0.4553, -0.2691,  ...,  0.2960,  0.2319, -0.4783]],
       grad_fn=<CatBackward0>)

In [25]:
#Lets Implement Dropout -
dropout=nn.Dropout(0.40)
final_embed=dropout(z)
final_embed

tensor([[-0.6754, -0.0000, -1.4774,  ...,  0.0000,  0.3866, -0.7972],
        [ 1.6278, -0.7589, -0.0000,  ...,  0.0000,  0.0000, -0.7972],
        [-0.6754, -0.0000, -1.4774,  ...,  0.4933,  1.8900, -0.0000],
        ...,
        [-1.7215,  1.4335, -1.8978,  ...,  0.4933,  0.3866, -0.0000],
        [ 1.6278, -0.0000, -0.4485,  ...,  0.4933,  0.3866, -0.7972],
        [ 1.6278, -0.7589, -0.0000,  ...,  0.4933,  0.3866, -0.7972]],
       grad_fn=<MulBackward0>)

In [26]:
#Lets Create an Feed Forward Neural Network -
class FeedForwardNN(nn.Module):

    def __init__(self, embedding_dim, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x
len(cont_features)

5

In [27]:
torch.manual_seed(100)
model=FeedForwardNN(embedding_dim,len(cont_features),1,[100,50],p=0.4)
model

FeedForwardNN(
  (embeds): ModuleList(
    (0): Embedding(15, 8)
    (1): Embedding(5, 3)
    (2): Embedding(2, 1)
    (3): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=19, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)

### Define Loss And Optimizer

In [28]:
loss_function=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
print(df.shape)
print(cont_values.shape)
print(cont_values)

(1201, 10)
torch.Size([1201, 5])
tensor([[   65.,  8450.,   856.,   854.,    19.],
        [   80.,  9600.,  1262.,     0.,    46.],
        [   68., 11250.,   920.,   866.,    21.],
        ...,
        [   66.,  9042.,  1188.,  1152.,    81.],
        [   68.,  9717.,  1078.,     0.,    72.],
        [   75.,  9937.,  1256.,     0.,    57.]])


In [29]:
#Lets Do the Train_test_split -
batch_size=1200
test_size=int(batch_size*0.15)
train_categorical=cat_features[:batch_size-test_size]
test_categorical=cat_features[batch_size-test_size:batch_size]
train_cont=cont_values[:batch_size-test_size]
test_cont=cont_values[batch_size-test_size:batch_size]
y_train=y[:batch_size-test_size]
y_test=y[batch_size-test_size:batch_size]
len(train_categorical),len(test_categorical),len(train_cont),len(test_cont),len(y_train),len(y_test)

(1020, 180, 1020, 180, 1020, 180)

In [30]:
#Lets Apply the Epoches -
epochs=5000
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model(train_categorical,train_cont)
    loss=torch.sqrt(loss_function(y_pred,y_train)) ### RMSE
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch number: 1 and the loss : 200496.75
Epoch number: 11 and the loss : 200493.46875
Epoch number: 21 and the loss : 200489.140625
Epoch number: 31 and the loss : 200482.640625
Epoch number: 41 and the loss : 200473.25
Epoch number: 51 and the loss : 200461.375
Epoch number: 61 and the loss : 200446.421875
Epoch number: 71 and the loss : 200429.375
Epoch number: 81 and the loss : 200407.953125
Epoch number: 91 and the loss : 200383.453125
Epoch number: 101 and the loss : 200355.296875
Epoch number: 111 and the loss : 200322.390625
Epoch number: 121 and the loss : 200291.40625
Epoch number: 131 and the loss : 200252.265625
Epoch number: 141 and the loss : 200206.109375
Epoch number: 151 and the loss : 200162.875
Epoch number: 161 and the loss : 200112.25
Epoch number: 171 and the loss : 200058.828125
Epoch number: 181 and the loss : 200005.9375
Epoch number: 191 and the loss : 199946.890625
Epoch number: 201 and the loss : 199881.703125
Epoch number: 211 and the loss : 199815.515625
Ep

In [ ]:
#Lets plot the Root mean square error -
plt.plot(range(epochs), final_losses)
plt.ylabel('RMSE Loss')
plt.xlabel('epoch');

In [32]:
#Lets Evaluate the Model -
y_pred=""
with torch.no_grad():
    y_pred=model(test_categorical,test_cont)
    loss=torch.sqrt(loss_function(y_pred,y_test))
print('RMSE: {}'.format(loss))

RMSE: 40083.9375


In [33]:
#Lets Make an Prediction -
data_verify=pd.DataFrame(y_test.tolist(),columns=["Test"])
data_predicted=pd.DataFrame(y_pred.tolist(),columns=["Prediction"])
data_predicted

,Prediction
0,121780.414062
1,208830.046875
2,147015.375000
3,235231.140625
4,220139.562500
5,212405.140625
6,149366.640625
7,305775.343750
8,155285.625000
9,387018.468750


In [34]:
#Lets arrange the Prediction Results -
final_output=pd.concat([data_verify,data_predicted],axis=1)
final_output['Difference']=final_output['Test']-final_output['Prediction']
final_output.head()

,Test,Prediction,Difference
0,130000.0,121780.414062,8219.585938
1,138887.0,208830.046875,-69943.046875
2,175500.0,147015.375000,28484.625000
3,195000.0,235231.140625,-40231.140625
4,142500.0,220139.562500,-77639.562500


In [35]:
#Lets Save the model -
torch.save(model,'HousePrice.pt')
torch.save(model.state_dict(),'HouseWeights.pt')

In [36]:
#Lets Load the model -
embs_size=[(15, 8), (5, 3), (2, 1), (4, 2)]
model1=FeedForwardNN(embs_size,5,1,[100,50],p=0.4)
model1.load_state_dict(torch.load('HouseWeights.pt'))
model1.eval()

FeedForwardNN(
  (embeds): ModuleList(
    (0): Embedding(15, 8)
    (1): Embedding(5, 3)
    (2): Embedding(2, 1)
    (3): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=19, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)